In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-09-07T14:47:48.839075-07:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.14.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1029-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 256
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm


from time import time

In [4]:
%watermark --iversions

numpy : 1.24.4
pandas: 1.5.3



In [5]:
ID = 'id'
TARGET = 'target'

In [6]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [7]:
y_train = train[TARGET].ravel()

train.drop([ID, TARGET], axis=1, inplace=True)
test.drop([ID], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")

(595212, 57),(892816, 57)


In [8]:
trains = []
for i in range(5):
    trains.append(train)

In [9]:
X_num_train_5 = np.vstack(trains)

In [10]:
X_num_train_5.shape

(2976060, 57)

In [12]:
columns = [f'Col{i}' for i in range(57)]

In [13]:
X_num_train_5 = pd.DataFrame(X_num_train_5, columns=columns)

In [14]:
y_train_5 = []
for i in range(5):
    y_train_5.append(y_train)

In [15]:
y_train_5 = np.hstack(y_train_5)

In [16]:
X_num_train_5.head()

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col47,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56
0,2.0,2.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,9.0,1.0,5.0,8.0,0.0,1.0,1.0,0.0,0.0,1.0
1,1.0,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,9.0,0.0,1.0,1.0,0.0,1.0,0.0
2,5.0,4.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,4.0,2.0,7.0,7.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,2.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0


In [17]:
X_num_train_5['target'] = y_train_5

In [18]:
X_num_train_5 = X_num_train_5.sample(frac=1).reset_index(drop=True)
X_num_train_5.head()

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,target
0,1.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,2.0,8.0,0.0,1.0,0.0,1.0,0.0,0.0,0
1,1.0,1.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,3.0,3.0,0.0,1.0,0.0,1.0,1.0,1.0,0
2,5.0,1.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0
3,2.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,4.0,6.0,0.0,1.0,1.0,0.0,0.0,0.0,0
4,2.0,1.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.0,2.0,11.0,0.0,0.0,0.0,1.0,1.0,0.0,0


In [19]:
X_num_train_5 = X_num_train_5.sample(frac=1).reset_index(drop=True)
X_num_train_5.head()

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,target
0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,9.0,0.0,1.0,1.0,0.0,0.0,0.0,0
1,1.0,1.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,2.0,7.0,0.0,0.0,0.0,1.0,0.0,1.0,0
2,5.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,3.0,11.0,0.0,1.0,0.0,1.0,0.0,0.0,1
3,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,3.0,5.0,0.0,1.0,0.0,1.0,0.0,0.0,1
4,5.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,4.0,14.0,0.0,1.0,0.0,1.0,0.0,0.0,0


In [20]:
X_num_train_5.to_csv('../input/xgtrain_aug_5.csv.gz', index=False, compression='gzip')